In [1]:
from google.colab import files
uploaded = files.upload()

Saving 09_db_centrocampista_filtered_top40pct.csv to 09_db_centrocampista_filtered_top40pct.csv


In [2]:
import pandas as pd
# Leemos el fichero
df = pd.read_csv('09_db_centrocampista_filtered_top40pct.csv', sep=",", encoding='utf-8')
# Mostramos las 5 primeras filas
df.head()

,Nombre completo,Lugar de nacimiento (país),Nacionalidad,Posición principal,Club actual,Proveedor,Fin de contrato,Fecha de fichaje,comprado_por,gk_diving,...,understat_matches,reactions,longpassing,dribbling,understat_assists,standingtackle,understat_minutes,penalties,Valor de mercado actual (numérico),Cluster
0,nico-o-reilly,46.0,209,0,54,6.0,30/06/2028,01/07/2024,NaN,-0.514870,...,NaN,-0.898181,0.288044,0.156993,NaN,0.271372,NaN,0.341909,7000000.0,2
1,Mateo Kovačić,6.0,93,0,54,6.0,30/06/2027,01/07/2023,2.261778,-0.418356,...,0.688955,1.581491,1.414456,1.128967,-0.736331,0.823563,0.002255,0.458979,25000000.0,1
2,İlkay Gündoğan,1.0,5,0,54,14.0,30/06/2026,23/08/2024,0.480667,-0.321842,...,1.272536,1.842509,1.414456,1.080369,3.420542,0.908515,1.597176,1.688220,7000000.0,1
3,kevin-de-bruyne,11.0,67,0,54,6.0,30/06/2025,30/08/2015,1.409943,-0.128814,...,-0.478207,2.364545,2.010792,1.031770,3.882417,0.568705,-0.329930,1.863826,27000000.0,1
4,Bernardo Mota Veiga de Carvalho e Silva,72.0,292,0,54,14.0,30/06/2026,01/07/2017,NaN,-0.418356,...,0.980746,1.973018,1.281937,1.177566,3.420542,0.823563,1.148131,0.751656,45000000.0,1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# Lista de columnas a excluir
exclude_columns = [
    'Lugar de nacimiento (país)', 'Nacionalidad', 'Club actual', 'Proveedor',
    'Fin de contrato', 'Fecha de fichaje', 'comprado_por',
    'Nombre completo', 'Posición principal'
]

# Obtener los valores únicos de la columna 'Cluster'
clusters = df['Cluster'].unique()

# Lista para guardar resultados por cluster
resultados = []

# Función para crear modelo y evitar retracing
def build_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# EarlyStopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

for cluster_id in clusters:
    df_cluster = df[df['Cluster'] == cluster_id].copy()

    # Eliminar columnas no deseadas y filas con nulos
    df_filtrado = df_cluster.drop(columns=exclude_columns, errors='ignore').dropna()


    # Separar X e y
    y = df_filtrado['Valor de mercado actual (numérico)']
    X = df_filtrado.drop(columns=['Valor de mercado actual (numérico)', 'Cluster'], errors='ignore')

    # Escalado
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    # División entrenamiento/prueba
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    if len(y_test) < 2:
        print(f"Cluster {cluster_id} tiene menos de 2 muestras en el test, se omite.")
        continue

    # Crear y entrenar modelo
    model = build_model(X_train.shape[1])
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=0
    )

    # Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Invertir escalado
    y_train_original = scaler_y.inverse_transform(y_train).ravel()
    y_test_original = scaler_y.inverse_transform(y_test).ravel()
    y_train_pred_original = scaler_y.inverse_transform(y_train_pred).ravel()
    y_test_pred_original = scaler_y.inverse_transform(y_test_pred).ravel()

    # Métricas
    r2_train = r2_score(y_train_original, y_train_pred_original)
    r2_test = r2_score(y_test_original, y_test_pred_original)
    mae_test = mean_absolute_error(y_test_original, y_test_pred_original)
    rmse_test = np.sqrt(mean_squared_error(y_test_original, y_test_pred_original))
    ratio_overfitting = r2_test / r2_train if r2_train != 0 else np.nan
    n_muestras = df_filtrado.shape[0]

    resultados.append({
        'Posición': 'Modelo por cluster',
        'Clúster': cluster_id,
        'N Muestras': n_muestras,
        'R2 Train': r2_train,
        'R2 Test': r2_test,
        'MAE Test': mae_test,
        'RMSE Test': rmse_test,
        'Ratio Overfitting': ratio_overfitting
    })

# Mostrar tabla de resultados
print(f"{'Posición':<20} {'Clúster':<10} {'N Muestras':<12} {'R2 Train':<10} {'R2 Test':<10} {'MAE Test':<10} {'RMSE Test':<10} {'Ratio Overfitting'}")
for res in resultados:
    print(f"{res['Posición']:<20} {res['Clúster']:<10} {res['N Muestras']:<12} {res['R2 Train']:<10.4f} {res['R2 Test']:<10.4f} {res['MAE Test']:<10.4f} {res['RMSE Test']:<10.4f} {res['Ratio Overfitting']:.4f}")


Epoch 49: early stopping
Restoring model weights from the end of the best epoch: 44.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Epoch 12: early stopping
Restoring model weights from the end of the best epoch: 7.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Epoch 28: early stopping
Restoring model weights from the end of the best epoch: 23.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Posición             Clúster    N Muestras   R2 Train   R2 Test    MAE Test   RMSE Test  Ratio Overfitting
Modelo por cluster   2          56           0.7736     -0.8053    2647506.3568 3786377.0814 -1.0410
Modelo por cluster   1          83           0.0922     0.0071     10690550.8235 13409956.8642 0.0771
Modelo por cluster   0          291          0.5149     0.1801     4358702.1462 5457108.1012 0.3497
Modelo por cluster   3          15           0.7525     0.3453     29296434.6667 40186378.3230 0.4589


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Lista de columnas a excluir
exclude_columns = [
    'Lugar de nacimiento (país)', 'Nacionalidad', 'Club actual', 'Proveedor',
    'Fin de contrato', 'Fecha de fichaje', 'comprado_por',
    'Nombre completo', 'Posición principal'
]

# Obtener valores únicos de clusters
clusters = df['Cluster'].unique()

# Lista para resultados
resultados = []

for cluster_id in clusters:
    df_cluster = df[df['Cluster'] == cluster_id].copy()

    # Filtrar columnas y eliminar nulos
    df_filtrado = df_cluster.drop(columns=exclude_columns, errors='ignore').dropna()

    if df_filtrado.shape[0] < 10:
        print(f"Cluster {cluster_id} tiene muy pocos datos ({df_filtrado.shape[0]}), se omite.")
        continue

    # Separar X e y
    y = df_filtrado['Valor de mercado actual (numérico)']
    X = df_filtrado.drop(columns=['Valor de mercado actual (numérico)', 'Cluster'], errors='ignore')

    # Escalado
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    if len(y_test) < 2:
        print(f"Cluster {cluster_id} tiene menos de 2 muestras en test, se omite.")
        continue

    # Entrenar modelo
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    # Predicciones
    y_train_pred = modelo.predict(X_train)
    y_test_pred = modelo.predict(X_test)

    # Invertir escalado
    y_train_orig = scaler_y.inverse_transform(y_train.reshape(-1, 1)).ravel()
    y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()
    y_train_pred_orig = scaler_y.inverse_transform(y_train_pred.reshape(-1, 1)).ravel()
    y_test_pred_orig = scaler_y.inverse_transform(y_test_pred.reshape(-1, 1)).ravel()

    # Métricas
    r2_train = r2_score(y_train_orig, y_train_pred_orig)
    r2_test = r2_score(y_test_orig, y_test_pred_orig)
    mae_test = mean_absolute_error(y_test_orig, y_test_pred_orig)
    rmse_test = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_orig))
    ratio_overfitting = r2_test / r2_train if r2_train != 0 else np.nan

    resultados.append({
        'Modelo': 'Lineal',
        'Clúster': cluster_id,
        'N Muestras': df_filtrado.shape[0],
        'R2 Train': r2_train,
        'R2 Test': r2_test,
        'MAE Test': mae_test,
        'RMSE Test': rmse_test,
        'Ratio Overfitting': ratio_overfitting
    })

# Mostrar resultados
print(f"{'Modelo':<10} {'Clúster':<10} {'N Muestras':<12} {'R2 Train':<10} {'R2 Test':<10} {'MAE Test':<10} {'RMSE Test':<10} {'Ratio Overfitting'}")
for res in resultados:
    print(f"{res['Modelo']:<10} {res['Clúster']:<10} {res['N Muestras']:<12} {res['R2 Train']:<10.4f} {res['R2 Test']:<10.4f} {res['MAE Test']:<10.4f} {res['RMSE Test']:<10.4f} {res['Ratio Overfitting']:.4f}")


Modelo     Clúster    N Muestras   R2 Train   R2 Test    MAE Test   RMSE Test  Ratio Overfitting
Lineal     2          56           0.6909     -0.7917    3025704.8346 3772098.1095 -1.1459
Lineal     1          83           0.5245     0.3148     9784165.2185 11139924.7126 0.6002
Lineal     0          291          0.4719     0.2587     4185691.0575 5188696.1443 0.5483
Lineal     3          15           1.0000     0.9014     13503049.0646 15596027.0849 0.9014
